In [1]:
from sentence_transformers import SentenceTransformer

In [ ]:
from ast import literal_eval

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
md = pd.read_csv('... movies_metadata.csv') # dataset do tmdb com os filmes que existem tanto nele, quanto no imdb

In [ ]:
m = round(md['vote_count'].quantile(0.95))  # dica: pegar so filmes com vote_count quantile > 95 (so filmes mainstream)
C = round(md['vote_average'].mean(), 1)     # isso fara nosso sistema de rec recomendar só filme mainstream tbm
    
df = md[(md['vote_count'] >= m)][['id', 'title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres', 'overview']]

df.shape

In [ ]:
# features para ajudar > recomendacoes baseadas em keywords e elenco (creditos) daquele filme
credits = pd.read_csv('... /credits.csv')
keywords = pd.read_csv('... /keywords.csv')

credits.drop_duplicates(inplace = True)
keywords.drop_duplicates(inplace = True)

In [ ]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
df['id'] = df['id'].astype('int')

In [ ]:
df = df.merge(credits, on='id')
df = df.merge(keywords, on='id')

In [ ]:
df['cast'] = df['cast'].apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df['cast'] = df['cast'].apply(lambda x: x[:5] if len(x) >=5 else x)

In [ ]:
df['keywords'] = df['keywords'].apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

df['director'] = df['crew'].apply(literal_eval).apply(get_director)

In [ ]:
# Removing space between names in order to differentiate between different actors and directors

df['cast'] = df['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
df['director'] = df['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))

In [ ]:
# Changing the datatype to list to enable concatenation
df['director'] = df['director'].apply(lambda x: [i for i in x.split()])

# Creating a new column by concatenating all the relavant metadata info
df['metadata'] = df['genres'] + df['keywords'] + df['director'] + df['cast']

# Taking all the items in the metadata and joining it into a single string
df['metadata'] = df['metadata'].apply(lambda x: ' '.join(x))

In [5]:
model = SentenceTransformer('all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\joao.franca\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\joao.franca\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

C:\Users\joao.franca\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
meta_data = np.array(df['metadata'])
embeddings = model.encode(text_data, show_progress_bar=True)

In [ ]:
model.get_sentence_features

In [ ]:
# Almost 1/4th of our data (531/2255) would get truncated as it exceeds the max_seq_length of 384. But it shouldn't affect the performance too much.

i = [len(i) for i in text_data if len(i) > 384]
len(i)

In [ ]:
embeddings.shape
cos_sim_data = pd.DataFrame(cosine_similarity(embeddings))

In [ ]:
df = df.reset_index()
data = df[['title','genres','year']]
indices = pd.Series(df.index, index=df['title'])

In [ ]:
def get_recommendations(title, N = 30):
    idx = indices[title]
    sim_scores = list(enumerate(cos_sim_data[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:N+1]
    movie_indices = [i[0] for i in sim_scores]
    sim_scores = pd.DataFrame(sim_scores, columns = ['index', 'similarity_score']) 
    final_data = data.iloc[movie_indices]
    final_data = final_data.merge(sim_scores, left_index = True, right_on ='index')
    final_data['similarity_score'] = round(final_data['similarity_score']*100,2)
    del final_data['index']
    return final_data

In [ ]:
get_recommendations('The Godfather', 15)